In [ ]:
import socket
import base64
import json
import gradio as gr
import pygame
import pygame_gui

# Define the TCP/IP socket for the client

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Define the IP address and port of the server

In [ ]:
server_address = ('localhost', 2351)

# Define the Gradio interface for the user to input the prompts

In [ ]:
def riffusion_prompt():
    # Get the starting and ending prompts from the user
    prompt_start_input = input("Enter starting prompt: ")
    prompt_end_input = input("Enter ending prompt: ")

# Encode the prompts as JSON

In [ ]:
prompt_json = json.dumps({
        "start": prompt_start_input,
        "end": prompt_end_input
    })

# Send the prompts to the server via TCP

In [ ]:
client_socket.sendall(prompt_json.encode())

# Receive the music from the server via UDP

In [ ]:
music_data, server_address = client_socket.recvfrom(1024)

# Decode the response and return the audio file

In [ ]:
audio_bytes = base64.b64decode(music_data)
    with open("output.mp3", "wb") as f:
        f.write(audio_bytes)
    return "output.mp3"


# Define the Gradio interface for the user to rate the music

In [ ]:
def riffusion_rating():
    # Get the user's rating for the music
    rating_input = input("Enter rating (good/bad): ")

    # Send the rating to the server via TCP
    client_socket.sendall(rating_input.encode())

    # Receive the current vote count from the server via TCP
    vote_count_data = client_socket.recv(1024)

    # Decode the response and return the vote count
    vote_count = json.loads(vote_count_data)
    return vote_count

# Initialize Pygame and create the GUI window

In [ ]:
pygame.init()
pygame.display.set_caption("Riffusion Voting")
window_surface = pygame.display.set_mode((640, 480))
gui_manager = pygame_gui.UIManager((640, 480))

# Define the GUI elements

In [ ]:
prompt_start_input = pygame_gui.elements.UITextEntryLine(
    relative_rect=pygame.Rect((50, 50), (400, 50)),
    manager=gui_manager
)
prompt_end_input = pygame_gui.elements.UITextEntryLine(
    relative_rect=pygame.Rect((50, 110), (400, 50)),
    manager=gui_manager
)
generate_music_button = pygame_gui.elements.UIButton(
    relative_rect=pygame.Rect((50, 170), (150, 50)),
    text="Generate Music",
    manager=gui_manager
)
rating_input = pygame_gui.elements.UITextEntryLine(
    relative_rect=pygame.Rect((50, 250), (400, 50)),
    manager=gui_manager
)
submit_rating_button = pygame_gui.elements.UIButton(
    relative_rect=pygame.Rect((50, 310), (150, 50)),
    text="Submit Rating",
    manager=gui_manager
)
vote_count_label = pygame_gui.elements.UILabel(
    relative_rect=pygame.Rect((50, 390), (400, 50)),
    text="Votes: 0",
    manager=gui_manager
)

# Run the GUI loop

In [ ]:
clock = pygame.time.Clock()
is_running = True
while is_running:
    # Handle Pygame events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            is_running = False
        elif event.type == pygame.USEREVENT:
            if event.user_type == pygame_gui.UI_BUTTON_PRESSED:
                if event.ui_element == generate_music_button:
                    # Get the prompts from the GUI
                    prompt_start = prompt_start_input.text
                    prompt_end = prompt_end_input.text

                    # Generate the music using the prompts
                    audio_file = riffusion_prompt()

                    # Play the music in Pygame
                    pygame.mixer.music.load(audio_file)
                    pygame.mixer.music.play()
                elif event.ui_element == submit_rating_button:
                    # Get the rating from the GUI
                    rating = rating_input.text

                    # Submit the rating to the server
                    vote_count = riffusion_rating()

                    # Update the vote count label in the GUI
                    vote_count_label.set_text(f"Votes: {vote_count}")
        gui_manager.process_events(event)

# Draw the GUI

In [ ]:
gui_manager.update(clock.tick(60) / 1000.0)
    window_surface.fill((255, 255, 255))
    gui_manager.draw_ui(window_surface)
    pygame.display.update()


# Clean up Pygame

In [ ]:
pygame.quit()

# Disconnect from the server

In [ ]:
client_socket.close()

description 
This client code uses a single TCP/IP socket to communicate with the server, and implements the riffusion_prompt() and riffusion_rating() functions that allow the user to input prompts for music generation and to rate the music generated by the server. The riffusion_prompt() function sends the prompts to the server via TCP and receives the music from the server via UDP, and the riffusion_rating() function sends the rating to the server via TCP and receives the current vote count from the server via TCP.


The code also uses Pygame and Pygame GUI to create a simple graphical user interface (GUI) that allows the user to input the prompts and submit the ratings. The GUI includes a "Generate Music" button that calls the riffusion_prompt() function when pressed, and a "Submit Rating" button that calls the riffusion_rating() function when pressed. The GUI also includes a label that displays the current vote count.


The code uses a single TCP/IP socket to communicate with the server, and disconnects from the server when either the game concludes or the rate at which the audio gets to it is deemed too slow.